In [ ]:
import sys
import cobra
import cplex
import cobrakbase
#Put the path to ModelSEEDpy on your machine here
sys.path.append("/Users/chenry/code/ModelSEEDpy")
#import modelseedpy.fbapkg
from modelseedpy import GapfillingPkg, KBaseMediaPkg
from modelseedpy import FBAHelper, MSBuilder, ProteomeFittingPkg, FlexibleBiomassPkg, 

In [ ]:
kbase_api = cobrakbase.KBaseAPI()
model = kbase_api.get_from_ws("C4_Calvin_Zmays_Model",25427)
model.solver = 'optlang-cplex'
#Setting model to complete media
#media = kbase_api.get_from_ws("Carbon-D-Glucose","KBaseMedia")
kmp = KBaseMediaPkg(model)
kmp.build_package(None)
#kmp.build_package(media)
fbp = FlexibleBiomassPkg(model)
fbp.build_package({"bio_rxn_id":"bio1"})

#Reading reaction protein abundances
data = ""
with open('ReactionProtein.txt', 'r') as file:
    data = file.read()
lines = data.split("\n")
reaction_proteins = {}
headers = None
for line in lines:
    if headers == None:
        headers = line.split("\t")
    else:
        array = line.split("\t")
        for i in range(1, len(array)):
            if array[0] not in reaction_proteins:
                reaction_proteins[headers[i]] = dict()
            if headers[i] == "reactions":
                reaction_proteins[headers[i]][array[0]] = array[i].split(";")
            else:
                reaction_proteins[headers[i]][array[0]] = abs(float(array[i]))

#Reading reaction kcat values
data = ""
with open('KCats.txt', 'r') as file:
    data = file.read()
lines = data.split("\n")
reaction_kcat = {}
headers = None
for line in lines:
    if headers == None:
        headers = line.split("\t")
    else:
        array = line.split("\t")
        reaction_kcat[array[0]] = array[1]

#Reading measured reaction fluxes
data = ""
with open('MeasuredReaction.txt', 'r') as file:
    data = file.read()
lines = data.split("\n")
reaction_measures = {}
headers = None
for line in lines:
    if headers == None:
        headers = line.split("\t")
    else:
        array = line.split("\t")
        if array[0] not in reaction_measures:
                reaction_measures[array[1]] = dict()
        reaction_measures[array[1]][array[0]] = abs(float(array[2]))

#Applying flexible biomass constraints
fbp = FlexibleBiomassPkg(model)
fbp.build_package({"bio_rxn_id":"bio1"})
        
#Applying proteome fitting constraints
pmp = ProteomeFittingPkg(model)
pmp.build_package({
    "rxn_proteome":reaction_proteins["Mature_leaf"],
    "flux_values":reaction_measures["Mature_leaf"],
    "kcat_values":reaction_kcat,
    "prot_coef" : 0.0948,#Set to the fraction of the cell that is protein
    "totalflux" : 1,#Set to one if we're fitting flux magnitude rather than actual flux to flux measurements
    "kcat_coef" : 0.02,#kapp = kcat_coef * kcat
    "obj_kfit":1,#Coefficient on kcat fitting terms in objective function
    "obj_kvfit":1,#Coefficient on kinetic expression fitting terms in objective function
    "obj_vfit":1,#Coefficient on flux measurement terms in objective function
    "set_objective":1
})

#Optimizing
sol=model.optimize()
model.summary()

In [39]:
tissues = [
    "Mature_leaf","Zone_1","Zone_2","Zone_3","GermEmbryo_2_DAI","Pericarp_Aleurone_27_DAP",
    "Endosperm_Crown_27_DAP","Endosperm_12_DAP","Endosperm_10_DAP","Endosperm_8_DAP","Embryo_38_DAP",
    "Embryo_20_DAP","root_Stele","root_Cortex","root_EZ","root_MZ"
]

protein_fraction = {
    "root_MZ" : 0,
    "root_EZ" : 0,
    "root_Cortex" : 0,
    "root_Stele" : 0,
    "Embryo_20_DAP" : 0,
    "Embryo_38_DAP" : 0,
    "Endosperm_8_DAP" : 0,
    "Endosperm_10_DAP" : 0,
    "Endosperm_12_DAP" : 0,
    "Endosperm_Crown_27_DAP" : 0,
    "Pericarp_Aleurone_27_DAP" : 0,
    "GermEmbryo_2_DAI" : 0,
    "Zone_1" : 0,
    "Zone_2" : 0,
    "Zone_3" : 0,
    "Mature_leaf" : 0.0948
}

for tissue in tissues:
    










    

        



#Loading model
kbase_api = cobrakbase.KBaseAPI()
res = kbase_api.get_objects2({"objects": [kbase_api.process_workspace_identifiers(, )]})
media = kbase_api.get_from_ws(params["media_id"],params["media_workspace"])
model = kbase_api.get_from_ws('C4_HS_Zmays_Model',"Full_Plant_Reconstructions")
model.solver = 'cplex'

utilities = KBaseFBAUtilities(model,model,kbase_api,None,
    default_uptake = 100,default_excretion = 100,blacklist = [])

#Add binary reversibility constraints
utilities.add_reversibility_binary_constraints()        

#Create total flux constraints
utilities.add_total_flux_constraints()

#Add flexible biomass constraints
utilities.add_flexible_biomass_constraints()

#Adding measured flux error constraints
obj_vars = []
error_variables = {}
error_constraints = {}
for baserxn_id in reaction_measures:
    for dataset in reaction_measures[baserxn_id]:
        if dataset in datasets:
            #Pseudoplastic reactions should be labeled with a z compartment
            rxn_id = baserxn_id+str(datasets[dataset])
            if baserxn_id[9:10] == "z":
                rxn_id = baserxn_id[0:9]+"d"+str(datasets[dataset]+20)
            if rxn_id in model.reactions:
                error_variables[rxn_id+"_meas"] = utilities.cobramodel.problem.Variable(rxn_id+"_meas", lb=-1*utilities.COBRA_DEFAULT_UB,ub=utilities.COBRA_DEFAULT_UB)
                obj_vars.append(measure_coef*error_variables[rxn_id+"_meas"] ** 2)
                error_constraints[rxn_id+"_meas"] = utilities.cobramodel.problem.Constraint(
                    error_variables[rxn_id+"_meas"] + utilities.total_flux_variables[rxn_id],lb=reaction_measures[baserxn_id][dataset],ub=reaction_measures[baserxn_id][dataset],name=rxn_id+"_meas"
                )
                model.add_cons_vars(error_variables[rxn_id+"_meas"])
                model.add_cons_vars(error_constraints[rxn_id+"_meas"])

measure_coef = 100
kcat_coef = 1
kinetic_coef = 1
                
#Adding kinetic expression error constraints
kinetic_variables = {}
rxn_kinetic_var = {}
for complex_id in reaction_proteins:
    kinetic_variables[complex_id+"_k"] = utilities.cobramodel.problem.Variable(complex_id+"_k", lb=0,ub=1000000)
    model.add_cons_vars(kinetic_variables[complex_id+"_k"])
    for dataset in datasets:
        error_vars = []
        coef = -1*protein_fraction[dataset]*reaction_proteins[complex_id][dataset]
        if coef != 0:
            error_vars.append(coef*kinetic_variables[complex_id+"_k"])
        error_variables[complex_id+"_"+dataset+"_k"] = utilities.cobramodel.problem.Variable(complex_id+"_"+dataset+"_k", lb=-1*utilities.COBRA_DEFAULT_UB,ub=utilities.COBRA_DEFAULT_UB)
        model.add_cons_vars(error_variables[complex_id+"_"+dataset+"_k"])
        error_vars.append(1*error_variables[complex_id+"_"+dataset+"_k"])
        obj_vars.append(kinetic_coef*error_variables[complex_id+"_"+dataset+"_k"] ** 2)   
        for baserxn_id in reaction_proteins[complex_id]["reactions"]:
            baserxn_id = baserxn_id[0:10]
            rxn_id = baserxn_id[0:10]+str(datasets[dataset])
            if rxn_id in model.reactions:
                rxn_kinetic_var[rxn_id] = kinetic_variables[complex_id+"_k"]
                error_vars.append(utilities.total_flux_variables[rxn_id])
            rxn_id = baserxn_id[0:10]+str(datasets[dataset]+20)
            if rxn_id in model.reactions:
                rxn_kinetic_var[rxn_id] = kinetic_variables[complex_id+"_k"]
                error_vars.append(utilities.total_flux_variables[rxn_id])
        error_constraints[complex_id+"_"+dataset+"_k"] = utilities.cobramodel.problem.Constraint(add(error_vars),lb=0,ub=0,name=complex_id+"_"+dataset+"_k")
        model.add_cons_vars(error_constraints[complex_id+"_"+dataset+"_k"])
            
#Adding kcat expression error constraints
kcat_variables = {}
for rxnobj in model.reactions:
    rxn_id = rxnobj.id
    base_id = rxnobj.id[0:8]
    if base_id in reaction_kcat:
        for dataset in datasets:
            error_variables[rxn_id+"_kcat"] = utilities.cobramodel.problem.Variable(rxn_id+"_kcat", lb=0,ub=1000000)
            obj_vars.append(kcat_coef*error_variables[rxn_id+"_kcat"] ** 2)
            if rxn_id in rxn_kinetic_var:
                error_constraints[rxn_id+"_kcat"] = utilities.cobramodel.problem.Constraint(
                    error_variables[rxn_id+"_kcat"] + rxn_kinetic_var[rxn_id],lb=0.02*reaction_kcat[base_id],ub=0.02*reaction_kcat[base_id],name=rxn_id+"_kcat"
                )

#Setting objective to minimize the error variables
model.objective = model.problem.Objective(add(obj_vars), direction="min", sloppy=True)

#Printing LP file
with open('FitProteomeData.lp', 'w') as out:
    out.write(str(model.solver))

In [26]:


import cobra
from cobra import Model, Reaction, Metabolite
import pandas as pd
import cplex
import optlang
import cobrakbase
sys.path.append("/Users/chenry/code/ModelSEEDpy")
#import modelseedpy.fbapkg
from modelseedpy import MSBuilder,MSGenome

>b0001 RAST:NADH transhydrogenase (EC:_.-.--)KO:K00100

template = kbase_api(wsid,templateid)
genome = MSGenome.genome_from_protein_hash({"b0001":"AJHSAKJD","b0002":""})
model = MSBuilder.build_model(genome,template)




236.0
